<a href="https://colab.research.google.com/github/stmulugheta/20223-ML-DL-models/blob/master/webscraping_shabait.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WebScraping for "https://shabait.com"

Initial support: Maria Fisher: roubadilhar = I have created this work from Portugues roubar (still) + compartilhar (share) = RoubaDilhar 3:-) 

I have roubadilhado* this codes from this tutorial (author : Jawakar)
https://medium.com/@jawakarselvavinayagam/scrape-data-from-web-to-csv-in-minutes-dfbaa1e8775d

# importning Libraries

In [3]:
#!pip install autoscraper
#!pip install pandas
#! pip install git+https://github.com/openai/whisper.git

In [7]:

import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
from autoscraper import AutoScraper

In [9]:
shabait = "https://shabait.com/"
wanted_list = ['Recent Posts ' ]

In [10]:
wanted_list

['Recent Posts ']

In [11]:
scraper = AutoScraper()
result = scraper.build(shabait, wanted_list)
data = scraper.get_result_similar(shabait, grouped=True)
print(data)

{'rule_tiwx': ['Recent Posts', 'Newspaper - Haddas Ertra', 'Newspapers - Eritrea Haddas', 'Newspapers - Eritrea Alhaditha', 'Newspapers - Eritrea Profile', 'Recent Posts'], 'rule_wd9p': ['Recent Posts', 'Newspaper - Haddas Ertra', 'Newspapers - Eritrea Haddas', 'Newspapers - Eritrea Alhaditha', 'Newspapers - Eritrea Profile', 'Stay With Us', 'Recent Posts']}


 Python dictionary with dynamically assigned keys
{'rule_rxxa': ['Recent'], 'rule_5s6n': ['Categories']}


In [12]:
key = list(data.keys())
print(key)
#output
{'rule_3no9': ['Recent Posts', 'Newspaper - Haddas Ertra', 'Newspapers - Eritrea Haddas', 'Newspapers - Eritrea Alhaditha', 'Newspapers - Eritrea Profile', 'Recent Posts'], 'rule_d5xd': ['Recent Posts', 'Newspaper - Haddas Ertra', 'Newspapers - Eritrea Haddas', 'Newspapers - Eritrea Alhaditha', 'Newspapers - Eritrea Profile', 'Recent Posts']}
scraper.save('shabait')

['rule_tiwx', 'rule_wd9p']


In [13]:
# Give it a file path
scraper.save('shabait')

In [14]:
# specify the path
scraper.load("shabait")
result = scraper.get_result_similar("https://shabait.com/", group_by_alias=True)

In [ ]:
result

{'': ['Recent Posts',
  'Recent Posts',
  'Newspaper - Haddas Ertra',
  'Newspaper - Haddas Ertra',
  'Newspapers - Eritrea Haddas',
  'Newspapers - Eritrea Haddas',
  'Newspapers - Eritrea Alhaditha',
  'Newspapers - Eritrea Alhaditha',
  'Newspapers - Eritrea Profile',
  'Newspapers - Eritrea Profile',
  'Stay With Us',
  'Recent Posts',
  'Recent Posts']}

In [15]:
#!pip install pandas

In [16]:
import pandas as pd

df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in result.items() ]))
# Converting our DataFrame to CSV file
df.to_csv('shabait.csv')

In [17]:
df.head(10)

,
0,Recent Posts
1,Recent Posts
2,Newspaper - Haddas Ertra
3,Newspaper - Haddas Ertra
4,Newspapers - Eritrea Haddas
5,Newspapers - Eritrea Haddas
6,Newspapers - Eritrea Alhaditha
7,Newspapers - Eritrea Alhaditha
8,Newspapers - Eritrea Profile
9,Newspapers - Eritrea Profile


In [18]:
df.tail(10)

,
3,Newspaper - Haddas Ertra
4,Newspapers - Eritrea Haddas
5,Newspapers - Eritrea Haddas
6,Newspapers - Eritrea Alhaditha
7,Newspapers - Eritrea Alhaditha
8,Newspapers - Eritrea Profile
9,Newspapers - Eritrea Profile
10,Stay With Us
11,Recent Posts
12,Recent Posts


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0           13 non-null     object
dtypes: object(1)
memory usage: 232.0+ bytes


In [20]:
df.describe()

,
count,13
unique,6
top,Recent Posts
freq,4


# To be continued according to the need

In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:

# Identify the target website's address, i.e., URL
url = 'https://shabait.com/category/newspapers/haddas-ertra-news/'
# Add an user agent to aviod 403 ERROR
user_agent = "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.37"
# Create a response object to get the web page's HTML content
get_url = requests.get(url, headers={'User-Agent': user_agent})
# Create a beautiful soup object to parse HTML text with the help of the html.parser
soup = BeautifulSoup(get_url.text, 'html.parser')
# Check website's response
print(get_url)

<Response [200]>


In [23]:

# Use prettify() method to make the HTML be nicely formatted
# This line of code is optional, which is used to take a look at HTML
# print(soup.prettify()) 
     

In [30]:
# Creat a list to store data
article_list = []
# Find all the topics
topics = soup.find_all('div',{'<a href="https://shabait.com/category/newspapers/">Newspapers</a>'})
# Calculate the number of topics
topic_count = 0
# Calculate the number of pdf links
pdf_count = 0

for item in topics:
  topic_count += 1
  # Fetch title of the article
  title = item.find('h2').find('a').text
  # Fetch the pdf link
  try:
    download_link = item.find('div',{'/html/body'}).find('a').get('href')   
  except:
    pass 
  # Get effective link
  if download_link.endswith('pdf') and download_link.startswith('http') :  
    ready_to_download = 'Y'  
    pdf_count += 1
  else:
    ready_to_download = 'N'
  # Creat a dict to stor article info
  article = {'title': title,
             'ready_to_download': ready_to_download,
             'pdf_link': download_link}
  
  article_list.append(article)
print(f'There are a total of {topic_count} posted on the website, {pdf_count} of which have effective pdf link') 

There are a total of 0 posted on the website, 0 of which have effective pdf link
